In [1]:


in_name = 'manuscript_covariates_4_final'
out_name = 'manuscript_covariates_5_final'


In [31]:
import psycopg2

import sys
sys.path.append( '..' )
import time
import importlib
import datetime
import pandas as pd
import numpy as np
import scipy.sparse
#import matplotlib.pyplot as plt
import Utils.dbutils as dbutils
import Utils.data_utils as data_utils
import Generators.CohortGenerator as CohortGenerator
import Generators.FeatureGenerator as FeatureGenerator
import config
local_imports = (
    dbutils,
    data_utils,
    CohortGenerator,
    FeatureGenerator,
    config
)
for i in local_imports:
    i = importlib.reload(i)

# database connection
username = config.PG_USERNAME
password = config.PG_PASSWORD
database_name = config.DB_NAME

#config_path = 'postgresql://{username}:{password}@{database_name}'.format(
config_path = 'postgresql://{database_name}'.format(database_name = database_name)

connect_args = {"host": '/var/run/postgresql/'} # connect_args to pass to sqlalchemy create_engine function

# schemas 
# schema_name = 'eol_test_ncjones' # all created tables will be created using this schema
cdm_schema_name = config.OMOP_CDM_SCHEMA # the name of the schema housing your OMOP CDM tables
print(f"cdm schema: {cdm_schema_name}")

    #username = username,
    #password = password,
   # database_name = database_name
#)

# schemas 
schema_name = 'eol_test_eheal' # all created tables will be created using this schema

# caching
reset_schema = False # if true, rebuild all data from scratch

# set up database, reset schemas as needed
db = dbutils.Database(config_path, schema_name, connect_args, cdm_schema_name)
# if reset_schema:
#     db.execute(
#         'drop schema if exists {} cascade'.format(schema_name)
#     )
# db.execute(
#     'create schema if not exists {}'.format(schema_name)
# )

cdm schema: cdm_6871_21


# Get Negative Controls

In [3]:
%%time
username='cdm_6871_21'
password='MLHC2021'
database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)
fibro_concepts = pd.read_sql("""SELECT * FROM cdm_6871_21.concept
                                 WHERE concept_name ILIKE '%fibrocystic%breast'""", conn)



CPU times: user 217 ms, sys: 39.6 ms, total: 257 ms
Wall time: 9.3 s


In [4]:
hernia_concepts = pd.read_sql("""SELECT * FROM cdm_6871_21.concept WHERE (concept_name ILIKE '%hernia%') 
                                AND (concept_name ILIKE ANY (ARRAY ['%ventral%', '%inguinal%', '%intestin%', '%lumb%', '%diaphragm%', '%umbilic%', '%epigastri%', '%hiat%', '%inci%']))""", conn)


In [5]:
fracture_concepts = pd.read_sql("""SELECT * FROM cdm_6871_21.concept
                                 WHERE concept_name ILIKE '%fracture%'""", conn)


### Remove highlighted terms

In [6]:
hernia_concepts = hernia_concepts[hernia_concepts.concept_name !='Diaphragmatic hernia']
hernia_concepts = hernia_concepts[hernia_concepts.concept_name !='Obstructed diaphragmatic hernia']
hernia_concepts = hernia_concepts[hernia_concepts.concept_name !='Congenital diaphragmatic hernia']
hernia_concepts = hernia_concepts[hernia_concepts.concept_name !='Congenital hiatus hernia']

In [7]:
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Fracture malunion - shoulder']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Open fracture of one rib']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Fracture malunion - forearm']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Malunion of fracture of proximal femur']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Late effect of fracture of skull AND/OR face bones']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Late effect of fracture of cervical vertebra']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Delayed union of fracture of foot']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Late effect of fracture of neck of femur']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Late effect of intracranial injury without skull fracture']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Late effect of fracture of spine AND/OR trunk without spinal cord lesion']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Fracture of tooth']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Sequelae of fracture of femur']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Disorder due to and following fracture of upper limb']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Late effect of fracture of thoracic vertebra']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Late effect of fracture of lower extremities']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Late effect of fracture of lumbar vertebra']
fracture_concepts =fracture_concepts[fracture_concepts.concept_name !='Delayed union of fracture']




In [8]:
# pd.DataFrame(list(fracture_concepts.concept_name),columns = ['fracture']).to_csv('Negative_Control_files/all_fracture.csv', index=False)
# pd.DataFrame(list(hernia_concepts.concept_name),columns = ['hernia']).to_csv('Negative_Control_files/all_hernia.csv', index=False)
# pd.DataFrame(list(fibro_concepts.concept_name),columns = ['fibro']).to_csv('Negative_Control_files/all_fibro.csv', index=False)



In [8]:
%%time

# get all conditions
sql = """
    select 
        a.condition_concept_id as concept_id,
        a.condition_start_date as condition_start_date,
        a.condition_occurrence_id as condition_occurrence_id,
        p.person_id as person_id
    from
        {omop_schema}.condition_occurrence a
    inner join
        {omop_schema}.person p
    on 
        a.person_id = p.person_id
    where 
        p.gender_source_value = 'F'

""".format(omop_schema='cdm_6871_21')
#.format(omop_schema=config.OMOP_CDM_SCHEMA,)
#condition_df = db.query(sql)
condition_df = pd.read_sql(sql,conn)
condition_df.set_index(["condition_occurrence_id"])
display(condition_df.head())

,concept_id,condition_start_date,condition_occurrence_id,person_id
0,443744,2015-10-09,170574397,13
1,437833,2014-12-09,110201808,13
2,80809,2015-12-10,183780400,13
3,254761,2014-04-28,69378391,13
4,79106,2014-06-23,78917016,13


CPU times: user 5min 7s, sys: 1min 21s, total: 6min 29s
Wall time: 12min 38s


In [9]:


sql_cmd = 'select * from cdm_6871_21.' + in_name
cohort_raw = pd.read_sql(sql_cmd, conn)
cohort_raw.head()

,level_0,index,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,visit_occurrence_id,...,t_uti_bin,t_neph_bin,AE_any,cbc_present,urine_test_present,years_since_diagnosis,followup_time,less_15,less_30,less_90
0,0,3828990,465004090,101,81902,2019-04-28,2019-04-28,0,0,914389,...,0,0,0,0,0,2,53913600000000000,0,0,0
1,1,3828990,148574471,481,81902,2015-06-09,2015-06-09,0,0,925277,...,0,0,0,0,0,6,113702400000000000,0,0,0
2,2,3828990,247942248,481,37018854,2016-09-13,2016-09-13,0,0,925268,...,0,0,0,0,0,5,113702400000000000,0,0,0
3,3,3828990,455153289,658,81902,2019-06-01,2019-06-01,0,0,929640,...,0,0,0,0,1,2,57715200000000000,0,0,0
4,4,115391,565954572,658,37018854,2021-01-14,2021-01-14,0,0,929764,...,0,0,0,0,1,0,57715200000000000,0,0,0


In [10]:
current_person_ids =cohort_raw.person_id.unique()

In [11]:
%%time
# Fibro
condition_ids = fibro_concepts.concept_id.unique()
fibro_conditions = condition_df[condition_df.concept_id.isin(condition_ids) & condition_df.person_id.isin(current_person_ids)]
fibro_conditions = fibro_conditions[["concept_id", "condition_start_date", "person_id"]]
#excluded_conditions = excluded_conditions.rename(columns={"condition_start_date":"date"})

# hernia
condition_ids =hernia_concepts.concept_id.unique()
hernia_conditions = condition_df[condition_df.concept_id.isin(condition_ids) & condition_df.person_id.isin(current_person_ids)]
hernia_conditions = hernia_conditions[["concept_id", "condition_start_date", "person_id"]]
#excluded_conditions = excluded_conditions.rename(columns={"condition_start_date":"date"})

# Fracture
condition_ids = fracture_concepts.concept_id.unique()
fracture_conditions = condition_df[condition_df.concept_id.isin(condition_ids) & condition_df.person_id.isin(current_person_ids)]
fracture_conditions = fracture_conditions[["concept_id", "condition_start_date", "person_id"]]
#excluded_conditions = excluded_conditions.rename(columns={"condition_start_date":"date"})


CPU times: user 8.23 s, sys: 665 ms, total: 8.89 s
Wall time: 8.89 s


In [12]:
outcomes = cohort_raw.copy()

outcomes = outcomes.sort_values(by=["person_id", "condition_start_date"], ascending=True)

outcomes['fibro_1_mon_outcome'] = False
outcomes['fibro_3_mon_outcome'] = False
outcomes['fibro_6_mon_outcome'] = False

outcomes['hernia_1_mon_outcome'] = False
outcomes['hernia_3_mon_outcome'] = False
outcomes['hernia_6_mon_outcome'] = False

outcomes['fracture_1_mon_outcome'] = False
outcomes['fracture_3_mon_outcome'] = False
outcomes['fracture_6_mon_outcome'] = False


In [13]:
%%time
# Hernias
# 
hernia_codes_30 = []
hernia_codes_90 = []
hernia_codes_180 = []

c= 0.
for index, row in outcomes.iterrows():
    flag30 = False
    flag90 = False
    flag180 = False
    
    
    p = row.person_id
    d = row.condition_start_date.date()
    hernia_df = hernia_conditions[hernia_conditions.person_id == p]
    dates = hernia_df.condition_start_date.unique()
    c += 1
    l = len(outcomes)
    if c % 5000 == 0:
        print("Iter: "+ str(c / l))
    for event_date in dates:
        if (event_date-d).days < 30 and (event_date-d).days > 0:
            outcomes.loc[index, 'hernia_1_mon_outcome'] = True
            if flag30 ==False:
                hernia_codes_30.append(list(set(list(hernia_df[hernia_df.condition_start_date == event_date].concept_id))))
                flag30=True
        if (event_date-d).days < 90 and (event_date-d).days > 0:
            outcomes.loc[index, 'hernia_3_mon_outcome'] = True
            if flag90 ==False:
                hernia_codes_90.append(list(set(list(hernia_df[hernia_df.condition_start_date == event_date].concept_id))))
                flag90=True
        if (event_date-d).days < 180 and (event_date-d).days > 0:
            outcomes.loc[index, 'hernia_6_mon_outcome'] = True
            if flag180 ==False:
                hernia_codes_180.append(list(set(list(hernia_df[hernia_df.condition_start_date == event_date].concept_id))))
                flag180 = True
            

Iter: 0.08316976612661765
Iter: 0.1663395322532353
Iter: 0.24950929837985295
Iter: 0.3326790645064706
Iter: 0.41584883063308825
Iter: 0.4990185967597059
Iter: 0.5821883628863236
Iter: 0.6653581290129412
Iter: 0.7485278951395589
Iter: 0.8316976612661765
Iter: 0.9148674273927941
Iter: 0.9980371935194118
CPU times: user 46.8 s, sys: 464 ms, total: 47.2 s
Wall time: 47.1 s


In [14]:
%%time
hernia_codes_30_all =  [item for sublist in hernia_codes_30 for item in sublist]
hernia_codes_90_all =  [item for sublist in hernia_codes_90 for item in sublist]
hernia_codes_180_all =  [item for sublist in hernia_codes_180 for item in sublist]

hernia_30 = pd.DataFrame(columns = ['concept_id','name'])
for ci in hernia_codes_30_all:
    hernia_30=hernia_30.append({'concept_id':ci,'name':hernia_concepts[hernia_concepts.concept_id==ci].concept_name.values[0].replace(',', '')}, ignore_index = True)
hernia_90 = pd.DataFrame(columns = ['concept_id','name'])
for ci in hernia_codes_90_all:
    hernia_90=hernia_90.append({'concept_id':ci,'name':hernia_concepts[hernia_concepts.concept_id==ci].concept_name.values[0].replace(',', '')}, ignore_index = True)
hernia_180 = pd.DataFrame(columns = ['concept_id','name'])
for ci in hernia_codes_180_all:
    hernia_180=hernia_180.append({'concept_id':ci,'name':hernia_concepts[hernia_concepts.concept_id==ci].concept_name.values[0].replace(',', '')}, ignore_index = True)
              
    
    

CPU times: user 2.28 s, sys: 14.8 ms, total: 2.29 s
Wall time: 2.28 s


In [15]:
%%time
# NEW *************************
# Fibro
# 
fibro_codes_30 = []
fibro_codes_90 = []
fibro_codes_180 = []

c= 0.
for index, row in outcomes.iterrows():
    flag30 = False
    flag90 = False
    flag180 = False
    
    p = row.person_id
    d = row.condition_start_date.date()
    fibro_df = fibro_conditions[fibro_conditions.person_id == p]
    dates = fibro_df.condition_start_date
    c += 1
    l = len(outcomes)
    if c % 10000 == 0:
        print("Iter: "+ str(c / l))
    for event_date in dates:
        if (event_date-d).days < 30 and (event_date-d).days > 0:
            outcomes.loc[index, 'fibro_1_mon_outcome'] = True
            if flag30 ==False:
                fibro_codes_30.append(list(set(list(fibro_df[fibro_df.condition_start_date == event_date].concept_id))))
                flag30=True
        if (event_date-d).days < 90 and (event_date-d).days > 0:
            outcomes.loc[index, 'fibro_3_mon_outcome'] = True
            if flag90 ==False:
                fibro_codes_90.append(list(set(list(fibro_df[fibro_df.condition_start_date == event_date].concept_id))))
                flag90=True
        if (event_date-d).days < 180 and (event_date-d).days > 0:
            outcomes.loc[index, 'fibro_6_mon_outcome'] = True
            if flag180 ==False:
                fibro_codes_180.append(list(set(list(fibro_df[fibro_df.condition_start_date == event_date].concept_id))))
                flag180=True
      
            

Iter: 0.1663395322532353
Iter: 0.3326790645064706
Iter: 0.4990185967597059
Iter: 0.6653581290129412
Iter: 0.8316976612661765
Iter: 0.9980371935194118
CPU times: user 43.5 s, sys: 295 ms, total: 43.8 s
Wall time: 43.7 s


In [16]:
fibro_codes_30_all =  [item for sublist in fibro_codes_30 for item in sublist]
fibro_codes_90_all =  [item for sublist in fibro_codes_90 for item in sublist]
fibro_codes_180_all =  [item for sublist in fibro_codes_180 for item in sublist]

fibro_30 = pd.DataFrame(columns = ['concept_id','name'])
for ci in fibro_codes_30_all:
    fibro_30=fibro_30.append({'concept_id':ci,'name':fibro_concepts[fibro_concepts.concept_id==ci].concept_name.values[0].replace(',', '')}, ignore_index = True)
fibro_90 = pd.DataFrame(columns = ['concept_id','name'])
for ci in fibro_codes_90_all:
    fibro_90=fibro_90.append({'concept_id':ci,'name':fibro_concepts[fibro_concepts.concept_id==ci].concept_name.values[0].replace(',', '')}, ignore_index = True)
fibro_180 = pd.DataFrame(columns = ['concept_id','name'])
for ci in fibro_codes_180_all:
    fibro_180=fibro_180.append({'concept_id':ci,'name':fibro_concepts[fibro_concepts.concept_id==ci].concept_name.values[0].replace(',', '')}, ignore_index = True)
              

In [17]:
%%time
# NEW *************************
fracture_codes_30 = []
fracture_codes_90 = []
fracture_codes_180 = []

c= 0.
for index, row in outcomes.iterrows():
    flag30 = False
    flag90 = False
    flag180 = False
    p = row.person_id
    d = row.condition_start_date.date()
    fracture_df = fracture_conditions[fracture_conditions.person_id == p]
    dates = fracture_df.condition_start_date
    c += 1
    l = len(outcomes)
    if c % 10000 == 0:
        print("Iter: "+ str(c / l))
    for event_date in dates:
        if (event_date-d).days < 30 and (event_date-d).days > 0:
            outcomes.loc[index, 'fracture_1_mon_outcome'] = True
            if flag30 ==False:
                fracture_codes_30.append(list(set(list(fracture_df[fracture_df.condition_start_date == event_date].concept_id))))
                flag30=True
        if (event_date-d).days < 90 and (event_date-d).days > 0:
            outcomes.loc[index, 'fracture_3_mon_outcome'] = True
            if flag90 ==False:
                fracture_codes_90.append(list(set(list(fracture_df[fracture_df.condition_start_date == event_date].concept_id))))
                flag90=True
        if (event_date-d).days < 180 and (event_date-d).days > 0:
            outcomes.loc[index, 'fracture_6_mon_outcome'] = True
            if flag180 ==False:
                fracture_codes_180.append(list(set(list(fracture_df[fracture_df.condition_start_date == event_date].concept_id))))
                flag180=True
      
            
            

Iter: 0.1663395322532353
Iter: 0.3326790645064706
Iter: 0.4990185967597059
Iter: 0.6653581290129412
Iter: 0.8316976612661765
Iter: 0.9980371935194118
CPU times: user 57.7 s, sys: 330 ms, total: 58.1 s
Wall time: 58.1 s


In [18]:
fracture_codes_30_all =  [item for sublist in fracture_codes_30 for item in sublist]
fracture_codes_90_all =  [item for sublist in fracture_codes_90 for item in sublist]
fracture_codes_180_all =  [item for sublist in fracture_codes_180 for item in sublist]

fracture_30 = pd.DataFrame(columns = ['concept_id','name'])
for ci in fracture_codes_30_all:
    fracture_30=fracture_30.append({'concept_id':ci,'name':fracture_concepts[fracture_concepts.concept_id==ci].concept_name.values[0].replace(',', '')}, ignore_index = True)
fracture_90 = pd.DataFrame(columns = ['concept_id','name'])
for ci in fracture_codes_90_all:
    fracture_90=fracture_90.append({'concept_id':ci,'name':fracture_concepts[fracture_concepts.concept_id==ci].concept_name.values[0].replace(',', '')}, ignore_index = True)
fracture_180 = pd.DataFrame(columns = ['concept_id','name'])
for ci in fracture_codes_180_all:
    fracture_180=fracture_180.append({'concept_id':ci,'name':fracture_concepts[fracture_concepts.concept_id==ci].concept_name.values[0].replace(',', '')}, ignore_index = True)
              
    

In [20]:
### Show counts

In [21]:
# pd.DataFrame(hernia_30['name'].value_counts()).to_csv('Negative_Control_files/hernia_30.csv')

# pd.DataFrame(hernia_90['name'].value_counts()).to_csv('Negative_Control_files/hernia_90.csv')

# pd.DataFrame(hernia_180['name'].value_counts()).to_csv('Negative_Control_files/hernia_180.csv')

In [22]:
# pd.DataFrame(fibro_30['name'].value_counts()).to_csv('Negative_Control_files/fibro_30.csv')

# pd.DataFrame(fibro_90['name'].value_counts()).to_csv('Negative_Control_files/fibro_90.csv')

# pd.DataFrame(fibro_180['name'].value_counts()).to_csv('Negative_Control_files/fibro_180.csv')

In [23]:
# pd.DataFrame(fracture_30['name'].value_counts()).to_csv('Negative_Control_files/fracture_30.csv')

# pd.DataFrame(fracture_90['name'].value_counts()).to_csv('Negative_Control_files/fracture_90.csv')

# pd.DataFrame(fracture_180['name'].value_counts()).to_csv('Negative_Control_files/fracture_180.csv')

In [19]:
outcomes['hernia_6_mon_outcome'].value_counts()

False    59739
True       379
Name: hernia_6_mon_outcome, dtype: int64

In [20]:
outcomes['fracture_6_mon_outcome'].value_counts()

False    58335
True      1783
Name: fracture_6_mon_outcome, dtype: int64

In [21]:
outcomes['fibro_6_mon_outcome'].value_counts()

False    59801
True       317
Name: fibro_6_mon_outcome, dtype: int64

In [22]:
outcomes['fibro_6_mon_outcome'].value_counts()

False    59801
True       317
Name: fibro_6_mon_outcome, dtype: int64

In [23]:
outcomes['fracture_6_mon_outcome'].value_counts()

False    58335
True      1783
Name: fracture_6_mon_outcome, dtype: int64

In [24]:
outcomes['hernia_6_mon_outcome'].value_counts()

False    59739
True       379
Name: hernia_6_mon_outcome, dtype: int64

In [25]:
list(outcomes.keys())[-9:]

['fibro_1_mon_outcome',
 'fibro_3_mon_outcome',
 'fibro_6_mon_outcome',
 'hernia_1_mon_outcome',
 'hernia_3_mon_outcome',
 'hernia_6_mon_outcome',
 'fracture_1_mon_outcome',
 'fracture_3_mon_outcome',
 'fracture_6_mon_outcome']

In [26]:
negative_controls = list(outcomes.keys())[-9:]

In [27]:
## Merge back

import pickle
outcomes_temp = outcomes[['condition_occurrence_id','person_id']+list(outcomes.keys())[-9:]]

cohort_new = cohort_raw.merge(outcomes_temp,how = 'outer',on = ['condition_occurrence_id','person_id'])
pickle.dump(cohort_new,open(f'{out_name}.pkl','wb'))

In [28]:
cohort_new.shape

(60118, 405)

In [29]:
final_cohort = cohort_new.drop(columns=['level_0'])

In [32]:

final_cohort.to_sql(out_name,con=db.engine, if_exists="replace", schema="cdm_6871_21")
cmd = 'grant select on table cdm_6871_21.{out_name} to cdm_6871_21'.format(out_name=out_name)
db.execute(cmd)

Executed 1 SQLs
